<a href="https://colab.research.google.com/github/SahanaPPatil/agrobot/blob/main/CNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import tensorflow as tf
import tensorflow_datasets as tfds
from tensorflow import keras
from tensorflow.keras import layers
import matplotlib.pyplot as plt
import numpy as np


In [ ]:
(ds_train, ds_info) = tfds.load(
    'plant_village',
    split='train',
    with_info=True,
    as_supervised=True
)

class_names = ds_info.features['label'].names
print(f"Number of classes: {len(class_names)}")
print("Example classes:", class_names[:10])


In [ ]:
plt.figure(figsize=(10, 10))
for i, (image, label) in enumerate(ds_train.take(9)):
    plt.subplot(3, 3, i + 1)
    plt.imshow(image)
    plt.title(class_names[label])
    plt.axis("off")
plt.show()


In [ ]:
def preprocess(image, label):
    image = tf.image.resize(image, (128, 128)) / 255.0
    return image, label

train_data = tfds.load('plant_village', split='train[:80%]', as_supervised=True)
test_data = tfds.load('plant_village', split='train[80%:]', as_supervised=True)

train_data = train_data.map(preprocess).shuffle(1000).batch(32)
test_data = test_data.map(preprocess).batch(32)


In [ ]:
model = keras.Sequential([
    layers.Conv2D(32, (3,3), activation='relu', input_shape=(128, 128, 3)),
    layers.MaxPooling2D(2,2),

    layers.Conv2D(64, (3,3), activation='relu'),
    layers.MaxPooling2D(2,2),

    layers.Conv2D(128, (3,3), activation='relu'),
    layers.MaxPooling2D(2,2),

    layers.Conv2D(256, (3,3), activation='relu'),
    layers.MaxPooling2D(2,2),

    layers.Flatten(),
    layers.Dense(128, activation='relu'),
    layers.Dropout(0.5),
    layers.Dense(38, activation='softmax')
])


In [ ]:
model.compile(
    optimizer='adam',
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

history = model.fit(
    train_data,
    validation_data=test_data,
    epochs=15
)


In [ ]:
model.save('my_model.keras')

In [ ]:
from tensorflow.keras.models import load_model
model = load_model('my_model.keras')
print("Model loaded successfully!")


In [ ]:
import json

with open("class_names.json", "w") as f:
    json.dump(class_names, f)



In [ ]:
test_loss, test_acc = model.evaluate(test_data)
print(f"\n Test Accuracy: {test_acc*100:.2f}%")


In [ ]:
for images, labels in test_data.take(1):
    preds = model.predict(images)
    plt.figure(figsize=(20,10))
    for i in range(10):
        plt.subplot(2, 5, i + 1)
        plt.imshow(images[i])
        true_label = class_names[labels[i]]
        pred_label = class_names[np.argmax(preds[i])]
        color = "green" if true_label == pred_label else "red"
        plt.title(f"Pred: {pred_label}\nTrue: {true_label}", color=color)
        plt.axis("off")
    plt.show()


In [ ]:
from tensorflow.keras.preprocessing import image

from google.colab import files
uploaded = files.upload()

for file_name in uploaded.keys():
    img_path = file_name
    img = image.load_img(img_path, target_size=(128, 128))
    img_array = image.img_to_array(img) / 255.0
    img_array = np.expand_dims(img_array, axis=0)

    prediction = model.predict(img_array)
    predicted_class = np.argmax(prediction)
    disease_name = class_names[predicted_class]
    confidence = np.max(prediction) * 100

    plt.imshow(img)
    plt.title(f"Predicted: {disease_name}\nConfidence: {confidence:.2f}%")
    plt.axis("off")
    plt.show()


In [ ]:
%%writefile app.py
import os
from flask import Flask, request, render_template
from werkzeug.utils import secure_filename

model = load_model('my_model.keras')

with open('class_names.json', 'r') as f:
    class_names = json.load(f)

app = Flask(__name__)
UPLOAD_FOLDER = "static/uploads"
app.config["UPLOAD_FOLDER"] = UPLOAD_FOLDER

ALLOWED = {'png', 'jpg', 'jpeg'}

def allowed_file(filename):
    return '.' in filename and filename.rsplit('.', 1)[1].lower() in ALLOWED

def predict_image(filepath):
    img = image.load_img(filepath, target_size=(128, 128))
    img_array = image.img_to_array(img) / 255.0
    img_array = np.expand_dims(img_array, axis=0)

    prediction = model.predict(img_array)[0]
    index = np.argmax(prediction)
    label = class_names[index]
    confidence = float(np.max(prediction)) * 100

    return label, confidence

@app.route("/")
def index():
    return render_template("index.html")

@app.route("/predict", methods=["POST"])
def upload():
    if "file" not in request.files:
        return "No file uploaded"

    file = request.files["file"]

    if file.filename == "":
        return "File has no name"

    if file and allowed_file(file.filename):
        filename = secure_filename(file.filename)
        filepath = os.path.join(app.config["UPLOAD_FOLDER"], filename)
        file.save(filepath)

        label, confidence = predict_image(filepath)
        return render_template("result.html",
                               filename=filename,
                               label=label,
                               confidence=confidence)

    return "File type not allowed"

@app.route('/uploads/<filename>')
def uploaded_file(filename):
    return f"/static/uploads/{filename}"

if __name__ == "__main__":
    app.run(host="0.0.0.0", port=5000)


In [ ]:

import os

os.makedirs("static/uploads", exist_ok=True)
os.makedirs("templates", exist_ok=True)

print("Folders created: static/uploads and templates/")


In [ ]:
%%writefile templates/index.html
<!DOCTYPE html>
<html>
<head>
    <title>Plant Disease Detector</title>
    <style>
        body { font-family: Arial; padding: 20px; }
        .card { width: 400px; margin: auto; padding: 20px; border: 1px solid #ccc; border-radius: 8px; }
        .btn { margin-top: 10px; padding: 10px 15px; background: #28a745; color: white; border: none; cursor: pointer; border-radius: 5px; }
    </style>
</head>
<body>
    <div class="card">
        <h2>Upload Leaf Image</h2>
        <form action="/predict" method="POST" enctype="multipart/form-data">
            <input type="file" name="file" accept="image/*" required><br><br>
            <button class="btn" type="submit">Predict Disease</button>
        </form>
    </div>
</body>
</html>


In [ ]:
%%writefile templates/result.html
<!DOCTYPE html>
<html>
<head>
    <title>Prediction Result</title>
    <style>
        body { font-family: Arial; padding: 20px; max-width: 800px; margin: auto; }
        .card { padding: 20px; border: 1px solid #ddd; border-radius: 8px; text-align: center; }
        img { max-width: 100%; height: auto; border-radius: 6px; margin-bottom: 12px; }
        .label { font-size: 1.1rem; margin-top: 8px; }
        a { display: inline-block; margin-top: 12px; color: #007bff; text-decoration: none; }
    </style>
</head>
<body>
    <h2>Prediction Result</h2>
    <div class="card">
        <!-- show the uploaded image -->
        <img src="{{ url_for('static', filename='uploads/' + filename) }}" alt="Uploaded Image">
        <div class="label"><strong>Predicted:</strong> {{ label }}</div>
        <div class="label"><strong>Confidence:</strong> {{ confidence }}%</div>
        <a href="{{ url_for('index') }}">Upload another image</a>
    </div>
</body>
</html>


In [ ]:
!pip install -q pyngrok flask tensorflow


In [ ]:
from pyngrok import ngrok
ngrok.set_auth_token("35gjGPfFAfPuVrBMWRgyzSCb5el_3SxnHh2jo3Qj7tdU8e7V4")
print("authtoken set")


In [ ]:
import subprocess, time, os, signal
from pyngrok import ngrok

try:
    ngrok.kill()
except Exception:
    pass

proc = subprocess.Popen(["python", "-u", "app.py"], stdout=subprocess.PIPE, stderr=subprocess.STDOUT)

time.sleep(2)

public_url = ngrok.connect(5000).public_url
print("Public URL:", public_url)
print("Flask process pid:", proc.pid)
